## Imports

In [ ]:
pip list

In [ ]:
pip uninstall gym -y

In [ ]:
pip install gym

In [ ]:
pip uninstall tensorflow -y

In [ ]:
pip install tensorflow==1.15.2

In [ ]:
from tensorflow.python.compiler import tensorrt as trt

In [ ]:
import stable_baselines
stable_baselines.__version__

In [ ]:
pip uninstall gym -y

In [ ]:
pip install gym==0.20.0

In [ ]:
import os
import gym
import time
import ml_monitor
import numpy as np
import matplotlib.pyplot as plt

from stable_baselines import DDPG, TD3
from stable_baselines.ddpg.policies import LnMlpPolicy
from stable_baselines.bench import Monitor
from stable_baselines.results_plotter import load_results, ts2xy
from stable_baselines.common.noise import AdaptiveParamNoiseSpec, NormalActionNoise
from stable_baselines.common.callbacks import BaseCallback

## Gym environment

In [ ]:
env = gym.make('LunarLanderContinuous-v2')

In [ ]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

In [ ]:
env.close()

## Training phase

### Callback function

In [ ]:
class SaveOnBestTrainingRewardCallback(BaseCallback):
    """
    Callback for saving a model (the check is done every ``check_freq`` steps)
    based on the training reward (in practice, we recommend using ``EvalCallback``).

    :param check_freq: (int)
    :param log_dir: (str) Path to the folder where the model will be saved.
      It must contains the file created by the ``Monitor`` wrapper.
    :param verbose: (int)
    """
    def __init__(self, check_freq: int, log_dir: str, verbose=1):
        super(SaveOnBestTrainingRewardCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = os.path.join(log_dir, 'best_model')
        self.best_mean_reward = -np.inf

    def _init_callback(self) -> None:
        my_monitor.monitor("testing_switch",0)
        my_monitor.monitor("training_switch",1)
        # Create folder if needed
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:

            # Retrieve training reward
            x, y = ts2xy(load_results(self.log_dir), 'timesteps')
            if len(x) > 0:
                # Mean training reward over the last 100 episodes
                mean_reward = np.mean(y[-100:])
                if self.verbose > 0:
                    print("Num timesteps: {}".format(self.num_timesteps))
                    print("Best mean reward: {:.2f} - Last mean reward per episode: {:.2f}".format(self.best_mean_reward, mean_reward))
                    my_monitor.monitor("num_timesteps",self.num_timesteps)
                    my_monitor.monitor("mean_reward",mean_reward)
                    
            # New best model
            if mean_reward > self.best_mean_reward:
                self.best_mean_reward = mean_reward
                my_monitor.monitor("best_mean_reward",self.best_mean_reward)
                # Saving best model
                if self.verbose > 0:
                    print("Saving new best model to {}".format(self.save_path))
                self.model.save(self.save_path)

        return True

In [ ]:
# Create log dir
log_dir = "machine-learning/logs/"
os.makedirs(log_dir, exist_ok=True)

In [ ]:
# Creating the environment
env = gym.make('LunarLanderContinuous-v2')
# Logs will be saved in log_dir/monitor.csv
env = Monitor(env, log_dir)

### Training the model

In [ ]:
# Add some param noise for exploration
param_noise = AdaptiveParamNoiseSpec(initial_stddev=0.1, desired_action_stddev=0.1)
# Create the callback: check every 1000 steps
callback = SaveOnBestTrainingRewardCallback(check_freq=1000, log_dir=log_dir)

In [ ]:
my_monitor = ml_monitor.Monitor()
my_monitor.start()

In [ ]:
# Because we use parameter noise, we should use a MlpPolicy with layer normalization
model = DDPG(LnMlpPolicy, env, param_noise=param_noise, verbose=False)
# Train the agent
model.learn(total_timesteps=int(2e6), callback=callback)

In [ ]:
model.save("machine-learning/models/LunarLander2e6")

## Testing phase

### Fast import

In [ ]:
pip install gym==0.20.0

In [1]:
import os
import gym
import time
import numpy as np
import matplotlib.pyplot as plt
import ml_monitor

from stable_baselines import DDPG, TD3
from stable_baselines.ddpg.policies import LnMlpPolicy
from stable_baselines.bench import Monitor
from stable_baselines.results_plotter import load_results, ts2xy
from stable_baselines.common.noise import AdaptiveParamNoiseSpec, NormalActionNoise
from stable_baselines.common.callbacks import BaseCallback

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/home/darias/.local/lib/python3.6/site-packages/stable_baselines/__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


### Loading model

In [ ]:
del model

In [2]:
env = gym.make('LunarLanderContinuous-v2')
model = DDPG.load("machine-learning/models/LunarLander1e6")





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.Dense instead.





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






### Agent environment

In [3]:
my_monitor = ml_monitor.Monitor()
my_monitor.start()

2022-02-10 02:59:59,252 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:00:00,254 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:00:01,255 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:00:02,257 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...


In [4]:
my_monitor.monitor("testing_switch",1)
my_monitor.monitor("training_switch",0)
episodes = 100
for episodes in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    my_monitor.monitor("testing_episodes",episodes)
    my_monitor.monitor("testing_score",score)
    print('Episode:{} Score:{}'.format(episodes, score))
    time.sleep(3)
env.close()

2022-02-10 03:00:03,258 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:00:04,260 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:00:05,261 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...


Episode:1 Score:243.22222832228906


2022-02-10 03:00:06,266 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:00:07,267 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:00:08,268 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:00:09,270 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:00:10,271 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:00:11,272 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...


Episode:2 Score:284.4380653959041


2022-02-10 03:00:12,274 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:00:13,275 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:00:14,276 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:00:15,278 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:00:16,279 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...


...

Episode:99 Score:278.15242125962527


2022-02-10 03:11:44,398 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:11:45,399 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:11:46,400 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:11:47,401 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:11:48,403 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...


Episode:100 Score:254.64923403693524


2022-02-10 03:11:49,404 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:11:50,405 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:11:51,406 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:11:52,408 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:11:53,409 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:11:54,410 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:11:55,411 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:11:56,413 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:11:57,414 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:11:58,416 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:11:59,417 [4759] INFO     ml_monitor

2022-02-10 03:13:16,526 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:13:17,527 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:13:18,528 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:13:19,530 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:13:20,532 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:13:21,533 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:13:22,534 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:13:23,536 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:13:24,537 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:13:25,539 [4759] INFO     ml_monitor.logging: Starting metrics logging thread...
2022-02-10 03:13:26,541 [4759] INFO     ml_monitor

In [ ]:
env.close()

## Ploting results

In [ ]:
from stable_baselines import results_plotter

# Helper from the library
results_plotter.plot_results([log_dir], 1e5, results_plotter.X_TIMESTEPS, "DDPG LunarLander")

In [ ]:
def moving_average(values, window):
    """
    Smooth values by doing a moving average
    :param values: (numpy array)
    :param window: (int)
    :return: (numpy array)
    """
    weights = np.repeat(1.0, window) / window
    return np.convolve(values, weights, 'valid')


def plot_results(log_folder, title='Learning Curve'):
    """
    plot the results

    :param log_folder: (str) the save location of the results to plot
    :param title: (str) the title of the task to plot
    """
    x, y = ts2xy(load_results(log_folder), 'timesteps')
    y = moving_average(y, window=50)
    # Truncate x
    x = x[len(x) - len(y):]

    fig = plt.figure(title)
    plt.plot(x, y)
    plt.xlabel('Number of Timesteps')
    plt.ylabel('Rewards')
    plt.title(title + " Smoothed")
    plt.show()


In [ ]:
plot_results(log_dir)